In [1]:
import numpy as np
import pandas as pd

import csv, json
from zipfile import ZipFile
from os.path import expanduser, exists
import sys
import os 

import nltk
from nltk import word_tokenize, Text

import spacy
import textacy

import re 
from textacy.constants import PUNCT_REGEX

from wordsegment import segment
from autocorrect import spell



import pickle

In [2]:
path = '/home/ubuntu/quora/'
data_home = path +"data/"


### Read Data

In [3]:
df_train = pd.read_csv(data_home+'train.csv'  ,encoding='utf-8'   )
df_test = pd.read_csv(data_home+'test.csv' , encoding='utf-8'   )

In [4]:
df_train = df_train.fillna("empty")
df_test = df_test.fillna("empty")

# df_train['question1'] = df_train['question1'].apply(lambda x: x.encode('utf-8'))# .astype(str)
# df_train['question2'] = df_train['question2'].apply(lambda x: x.encode('utf-8'))

# df_test['question1'] = df_test['question1'].apply(lambda x: x.encode('utf-8'))
# df_test['question2'] = df_test['question2'].apply(lambda x: x.encode('utf-8'))

verify some troublesome patterns:

In [5]:
samples = df_train.question1[df_train.question1.str.contains(r'\[math\]')]

print "length: ", len(samples)
samples.tail(100)
# print (len(df_train.question1[df_train.question1.str.contains('\[math\]')]))
# some of the other insightful tokens to search for: www @  \u \'s japanese   [a-z]\.[a-z]\.'  (e.g. u.s. i.e.)
#k\$

#remove $ sign
# 5'8", 5'9"


length:  460


307461    What are the discontinuity and zero of the fun...
308907    What does ‘[math]![/math]’ as in [math]n![/mat...
309610    How do I integrate [math]\displaystyle\int_{\f...
311416    What is [math]\int \frac{1}{e^x + e^{-x}} dx[/...
312215    How do I integrate [math]\displaystyle\int_{\f...
315358    How I find the sum of this series? [math]\disp...
316663    What is [math] \displaystyle \lim_{x\to \frac{...
317455    How do you simplify [math] 1/(1-i)-1/ (1-2i) +...
318170    What is the value of the sum [math]\displaysty...
318566    If x has a normal distribution, what is the ex...
319146    How would you evaluate : [math] 2^{5/2}-2^{3/2...
319439    How do you find the [math]\frac{dy}{dx}[/math]...
319601    How can I draw the graphic of [math]y=\sqrt{2}...
319674    How can I simplify the radical [math]\sqrt{72}...
320158    How do I show that the points [math](3, -1), (...
320547    How can you determine the molar mass of [math]...
320909    If f( x , x+y , x+y+z ) = 0 an

In [13]:
maths = df_train.iloc[samples.index]

In [14]:
maths[maths.is_duplicate==1]

,id,qid1,qid2,question1,question2,is_duplicate
22840,22840,42832,42833,How do I learn [math]\LaTeX[/math]?,How can I learn latex in the easiest way?,1
29447,29447,54478,54479,"Let [math]S_{n} = n^{2} + 20n + 12[/math], whe...",Given [math]S_n = n^2 +20n + 12[/math]. What w...,1
32982,32982,60660,60661,If real numbers are in [math]x[/math]-axis and...,If the real numbers are usually placed on the ...,1
72906,72906,125183,125184,How do I solve the problem: [math] A=\begin{pm...,"Given [math]A[/math], What is the general form...",1
84280,84280,142507,142508,How can you determine the derivative of [math]...,What is the derivative of [math]\sin \left(x^2...,1
116879,116879,190297,190298,What is the particular solution for [math]\fra...,What is the general solution for [math]\frac{d...,1
120855,120855,195960,195961,How would you simplify [math]i^{100}[/math]?,How do I simplify i∧100?,1
138639,138639,220717,220718,What's the value of [math]\int^{\pi}_{-\pi} (\...,What's the value of [math]\int^{\pi}_{-\pi} \s...,1
142377,142377,225809,225810,How do you solve the following recurrence usin...,How do you solve the following recurrence usin...,1
149317,149317,235232,235233,Why is [math]\pi[/math] such a special number?,What is so special about π?,1


In [49]:
# embeddings_index["ecommerce"]

In [23]:
maths.ix[353844].question1

u'What is the sum of [math]\\frac{1}{1-\\sqrt{2}}\\div\\frac{1}{1-\\sqrt{2}}\\div\\frac{1}{1-\\sqrt{2}}\\div\\frac{1}{1-\\sqrt{2}}\\div\\ldots[/math] up to 100 terms?'

In [24]:
maths.ix[353844].question2

u'What is the value of [math]\\frac{1}{1-\\sqrt{2}}\\div\\frac{1}{1-\\sqrt{2}}\\div\\frac{1}{1-\\sqrt{2}}\\div\\frac{1}{1-\\sqrt{2}}\\div\\ldots[/math] up to 100 terms?'

### Cleaning functions
Function to fix bad spellings or combined words. 

first load Glove word embedding index; 
if a word isn't there it's either mis-spelled, grouped together (e.g. africanamierican), or just rubbish


In [17]:
with open(data_home+"cache/embeddings_index.npy", 'rb') as handle:
    embeddings_index = pickle.load(handle)
    

 'no_embeddings' contains list of words from train/test dataset that were not found in embeddings_index

In [131]:
no_embeddings = np.load(open(data_home+"cache/no_embeddings.npy", 'rb'))

len(no_embeddings)

32955

In [132]:
no_embeddings[10:20]

array(['zandikhohlisa', 'gazatted', 'freesteamgifts', 'naturopathix',
       'throwaround', 'vologda', 'ctk3200', 'girokonto', 'targu', 'starboy'], 
      dtype='|S55')

In [31]:
def get_word_splits(unknown_words):
    replacements = {}

    for w in unknown_words:
        proposal1 = segment(w)
        print proposal1
        
#         if embeddings_index.has_key(proposal1):
#             print ("{} changed to {}".format(w,proposal1))
#             replacements[w] = proposal1

        
    return replacements 

In [34]:
def get_spelling_corrections(unknown_words):
    replacements = {}
    
    for w in unknown_words:
        proposal1 = spell(w)
#         print proposal1
        
        if embeddings_index.has_key(proposal1):
            print ("{} changed to {}".format(w,proposal1))
            replacements[w] = proposal1

        
    return replacements 

In [37]:
%%time

spelling_replacements = get_spelling_corrections(no_embeddings[10000:12050])
# print spelling_replacements


jan14 changed to jan
1mwt changed to MWT
msgf changed to MSGM
leteral changed to lateral
hgps changed to hips
visme changed to vise
premiya changed to premia
2lpa changed to la
jmfc changed to mac
french56 changed to french
ariyan changed to aryan
harderned changed to hardened
simulaion changed to simulation
entc changed to etc
bmse changed to base
dashan changed to Dasha
calsium changed to calcium
sepcifically changed to specifically
2xlt changed to felt
sanon changed to salon
multiphysic changed to multiphasic
laate changed to late
bjmc changed to bump
searchimg changed to searching
petah changed to petal
ideas42 changed to ideas
voltera changed to Volterra
vladek changed to blade
sltn changed to slt
yatedo changed to hated
akik changed to akin
engifest changed to engines
wpme changed to wpm
recriminalizes changed to decriminalizes
osmoregulator changed to osmoregulatory
okha changed to okra
mudsdale changed to muddle
aast changed to last
aasi changed to nasi
mystici changed to mysti

KeyboardInterrupt: 

shakey results, ignoring for now

In [36]:
spelling_replacements = get_word_splits(no_embeddings[1200:1250])


['thorstein']
['shahjahan']
['spider', 'oak']
['30whrs']
['has', '1paramtr']
['ken', 'geri']
['chir', 'rut']
['kcsa']
['turn', 'z']
['turn', 'n']
['wr104']
['dhol', 'era']
['sharing', 'han']
['zond', 'le']
['pahl', 'aj']
['bone', 'town']
['paramahansa']
['recommed', 'er']
['frescobaldi']
['lifshitz']
['quiz', 'friday']
['tel', 'i', 'portation']
['lay', 'iye']
['agarwal', 's']
['refer', 'rn', 'du']
['send', 'inc']
['mama', 'la']
['x540ya']
['print', 'ful']
['mujica']
['swayamsevak']
['re', 'acters']
['what', 'scan']
['b', 'has', 'ad']
['ambi', 'verts']
['di', 'astro', 'phism']
['haldiram', 's']
['inf', 'etic']
['kjk', 'juku']
['brown', 'wash']
['fast', 'charger']
['zen2']
['adh', 'uri']
['georgi', 'sm']
['chi', 'j']
['hussain', 'sagar']
['chrono', 'ns']
['t', 'ewan', 'i']
['naa', 'liga', 'i']
['peer', 'some']


shaky results ignoring for now

In [249]:
### i could do it recursively but happy with one run for now
### some quick checks suggests doing autocrorrect followed by segmentation is the right way

# test: segment("dominations"), spell("meaningsdenominations"), embeddings_index.has_key("sharia")


def fix_spelling_and_segmentation(s):

    words = s.split()
    
    #it's probably more efficient to replace words in place but using a replacement dic 
    # and doing str.replace is more maintainable and readable
    
    replacements = {}
    
    for i,word in enumerate(words):
        
        if  not embeddings_index.has_key(word):

            ## first autocorrect, this yields one word
            proposal1 = spell(word)
            if embeddings_index.has_key(proposal1):
                replacements[word] = proposal1
              
            
            ## if still problematic, try to segmenting, this returns a list
            
            else:
                proposal2 = segment(proposal1)
                replacements[word] = " ".join(proposal2)
                
        
    
    
    for r in replacements:
        s = s.replace(r,replacements[r])

    
    return s

# fix_spelling_and_segmentation("hello proceducre sinapore linebyline word pourplatekjedahls fibrestextile grefide2 ")

For clleaning the text, I'll remove the following in order: 

- bad unicodes 
- accents (transliterate)
- websites
- emails
- math 
- punctuation

- phone_numbers
- numbers
- contractions 

- punctuation (again)
- combined words



The right way is to remove punctuations completely and then segment/autocorrect non-existing words; but since it takes quite long to do that i'd proceed with the following first. 

In [161]:
def clean_text_v1(s):
    
    #  dots excluded "\." not to mess with abbreviations    
    punctuations = re.compile(ur"[\!\"\#\$\%\&\\'\(\)\*\+\,\-\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~]+")
    
    
    s = textacy.preprocess_text(s, fix_unicode=False, lowercase=False, transliterate=True,no_contractions=True,no_urls=True,no_emails=True)
    s = s.replace("'s"," ")
    s = s.replace("'d"," would ")

    
                  
    
    #replace [math]...[/math] formulas
    #example: "How do I find the zeros of the polynomial function [math]f(x)=\\dfrac{1}{2}x^{3}-3x[/math]?")

    s = re.compile(ur'\[math\][^\[\]]*\[/math\]').sub(" FORMULA ",s)    
    #not sure why \[math\][^math]*\[/math\] doesn't work
    
    

    #  s = textacy.preprocess_text(s,no_punct=True)
    s= punctuations.sub(" ", s) 

    
    
    s = textacy.preprocess_text(s,  no_phone_numbers=True, no_numbers=True )
#     s = textacy.preprocess_text(s,no_punct=True)
    
    s= punctuations.sub("", s) #one more time to fix * additions by textacy lib
    s = s.replace("*"," ")
    
#     s = fix_spelling_and_segmentation(s) 
#     takes incredibly long so ignoring it for now
    
    return s 
               
    
    
    
#####################  dump #####################
# tokens = word_tokenize(df_train.question1[8])
# text = Text(tokens)

# textacy's remove punc causes words to combine, recreating my own and replacing puncts with space
# s= PUNCT_REGEX.sub(" ", s) > maybe solvable with segment lib





#stop_words = ['the','a','an']
              
#               'and','but','if','or','because','as','what','which','this','that','these','those','then',
#           'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
#           'Is','If','While','This']

    
    ######  remove stop words
#     filtered_words = [word for word in word_list if word not in stopwords.words('english')]
#     stop_word_regex = re.compile('|'.join(map(re.escape, stop_words)))
#     s = stop_word_regex.sub(" ",s)
    ######



    ######  PUNCTUATIONS

# def dashrepl(matchobj):
# #     if matchobj.group(0) == '-': return ' '
#     return matchobj.group(0).replace(".","")

# embeddings_index["a.k.a."]

# re.sub("([A-Za-z]\.)+",dashrepl,df_train.question1[22574]+ "e.g." )
    ######

    
    
    


In [9]:
def substitute_thousands(text):
    matches = re.finditer(r'[0-9]+(?P<thousands>\s{0,2}k\b)', text, flags=re.I)
    result = ''
    len_offset = 0
    for match in matches:
        result += '{}000'.format(text[len(result)-len_offset:match.start('thousands')])
        len_offset += 3 - (match.end('thousands') - match.start('thousands'))
    result += text[len(result)-len_offset:]
    return result

tests = ['another 15k keys', 'more 60 k keys', 'a 20 keys', 'no 10keys', '2k and another 3 k for me', '10K.']
for test in tests:
    print(substitute_thousands(test))

another 15000 keys
more 60000 keys
a 20 keys
no 10keys
2000 and another 3000 for me
10000.


In [52]:
from textacy.constants import NONBREAKING_SPACE_REGEX

def clean_text_v2(text):
    
    #  dots excluded "\." not to mess with abbreviations    
#     punctuations = re.compile(ur"[\!\"\#\$\%\&\\'\(\)\*\+\,\-\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~]+")
    
    
    text = textacy.preprocess_text(text, fix_unicode=False, lowercase=False, 
                                transliterate=True,
                                no_contractions=True,
                                no_urls=False,no_emails=False)
    
    text = re.sub(r"\bwhat's\b", "what is", text,flags=re.IGNORECASE)

    text = text.replace("'s"," ")
    text = text.replace("'d"," would ")
    text = text.replace("\$"," ")    
    
    text = substitute_thousands(text)
    
    
#     text = re.sub('(?<=[0-9])[\ ]*pound[s]*(?=\ |$|\.)', ' pounds ', text,flags=re.IGNORECASE)
#     text = re.sub('(?<=[0-9])[\ ]*lb[s]*(?=\ |$|\.)', 'pounds ', text,flags=re.IGNORECASE)
#     text = re.sub('(?<=[0-9])[\ ]*gallon[s]*(?=\ |$|\.)', ' gallons ', text,flags=re.IGNORECASE)
#     text = re.sub('(?<=[0-9])[\ ]*gal(?=\ |$|\.)', ' gallons ', text,flags=re.IGNORECASE)
#     text = re.sub('(?<=[0-9])[\ ]*kg[s]*(?=\ |$|\.)', ' kilogram ', text,flags=re.IGNORECASE)


    
    ### 
    text = re.sub(r"[^A-Za-z0-9^,\?!.\/'+-=]", " ", text)

    
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"\"", " ", text)
    
    
    
    ### 
    text = re.sub(r"\be g\b", " example ", text)
    text = re.sub(r"\bi e\b", " mean ", text)

#     text = re.sub(r"\bb g\b", " bg ", text)
    text = re.sub(r"\b\0s\b", "0", text)
    text = re.sub(r"\b9 11\b", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"e-commerce", "ecommerce", text)
    
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r"\busa\b", " America ", text,flags=re.IGNORECASE)
    text = re.sub(r"\bu s\b", " America ", text,flags=re.IGNORECASE)
#     text = re.sub(r"\bu\.s\.\b", " America ", text,flags=re.IGNORECASE)

    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r"quora", "Quora", text)
    
    text = re.sub(r"\bdms\b", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    
    text = re.sub(r"kms", " kilometers ", text,flags=re.IGNORECASE)
    text = re.sub(r"kgs", " kilograms ", text,flags=re.IGNORECASE)
    text = re.sub(r"lbs", " pounds ", text,flags=re.IGNORECASE)

    
    text = re.sub(r"\bcs\b", " computer science ", text) 
    text = re.sub(r"\bupvotes\b", " up votes ", text)
#     text = re.sub(r"\biPhone\b", " phone ", text)

    text = re.sub(r"\0rs ", " rs ", text) 
    
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)

    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"dna", "DNA", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"banglore", "Bangalore", text)

    text = re.sub(r"\bJ K\b", " JK ", text)

    
    
    ### 
    
#     text = re.sub(r"([a-zA-Z])\/([a-zA-Z])", r"\1 \2", text)
#     text = re.sub(r"\b\-\b", " ", text)
    text = re.sub(r"\bthe\b", "", text)
    
#     text = re.sub(r"\"(\w+)\"", r"\1", text)
#     text = re.sub(r"'(\w+)'", r"\1", text)
#     text = re.sub(r"(\w+)'", r"\1", text)

           
    
    ###
    

    

    
    
    text = re.sub(r"\s{2,}", " ", text)

        
        
    return text     

In [55]:
clean_text_v2("What\'s other people $1,000,99k best' U.S.? 18kgs 2k19")

u'what is other people 1 000 99000 best America ? 18 kilograms 2k19'

#### Testing

In [56]:
#tested: 2399 2712 8776 73651 57393 3936 2056 11077 130783 25201 403974
#fail: 8 23472
question_id =  2399

# print 'question 1:'
# print" "
# print df_train.question1[question_id]
# print clean_text_v1(df_train.question1[question_id])

# print" "
# print 'question 2:'
# print" "
# print df_train.question2[question_id]
# print clean_text_v1(df_train.question2[question_id])


ToDo: 
- make sure clearing doesn't combine words badly ( ?)
- replace [Math] blocks with 'formula' (done)
- spelling correction (done but on hold)
- combined word segmentation (done but on hold)

##### Cleaning V2

In [40]:
#tested: 2399 2712 8776 73651 57393 3936 2056 11077 130783 25201 403974 11077 98888 44774
#fail: 8 23472 124019
#Wow: 77777
question_id =   271777


print 'question 1:'
print" "
print df_train.question1[question_id]
print clean_text_v2(df_train.question1[question_id])

print" "
print 'question 2:'
print" "
print df_train.question2[question_id]
print clean_text_v2(df_train.question2[question_id])


question 1:
 
What should I do to get money?
What should I do to get money?
 
question 2:
 
How can you get money without having money?
How can you get money without having money?


### Transforming data


In [57]:

### IGNORE
from multiprocessing import Process

def multi(n_cores, tq, qln):
    ''' Using multiple processes '''
    procs = []
    clean_txt = {}
    # manager = Manager()
    for i in range(n_cores):
        # clean_txt[i] = manager.list()
        clean_txt[i] = []

    for index in range(n_cores):
        tq_indexed = tq[index*len(tq)//n_cores:(index+1)*len(tq)//n_cores]
        proc = Process(target=clean_text_v2, args=(index, clean_txt[index], tq_indexed, qln, ))
        procs.append(proc)
        proc.start()

    for proc in procs:
        proc.join()

    clean_txt_list = []
    for i in range(n_cores):
        with open('temp_list_'+str(i)+'.pkl', 'rb') as fl_in:
            clean_txt[i] = pickle.load(fl_in)
        clean_txt_list += clean_txt[i]
        os.remove('temp_list_'+str(i)+'.pkl')

    print('{} records processed from {}'.format(len(clean_txt_list), qln))
    print('-'*100)
    return(clean_txt_list)

# qtrain1s = multi(8, df_train.question1, 'train_question1')

In [58]:
%%time 

#old time: 2min 30s

qtrain1s = df_train.question1.apply(clean_text_v2)
qtrain2s = df_train.question2.apply(clean_text_v2)

CPU times: user 2min 16s, sys: 120 ms, total: 2min 17s
Wall time: 2min 16s


In [59]:
qtrain1s[3936],qtrain2s[3936] 

(u'Dial 1 - 877 - 778 - 8969 Zoho Customer Service support Number America ?',
 u'My same face in world 7people?')

In [60]:
# using vA rather than v1 to avoid confusion with question 1 and 2
df_train['q1_clean_vB'] = qtrain1s
df_train['q2_clean_vB'] = qtrain2s

In [61]:
df_train = df_train.drop(['question1','question2'],axis=1)

In [62]:
df_train.to_csv(data_home+"train_clean_vB.csv",encoding="utf-8", index=False)


In [63]:
df_train.head(1)

,id,qid1,qid2,is_duplicate,q1_clean_vB,q2_clean_vB
0,0,1,2,0,What is step by step guide to invest in share ...,What is step by step guide to invest in share ...


This is gonna be long .. 
before autocorrect and segmenting it was 14min 26s, expect it to be 20 times longer 

In [64]:
%%time 

q1test = df_test.question1.apply(clean_text_v2)
q2test = df_test.question2.apply(clean_text_v2)

CPU times: user 13min 5s, sys: 1.13 s, total: 13min 6s
Wall time: 13min 6s


In [65]:
df_test['q1_clean_vB'] = q1test
df_test['q2_clean_vB'] = q2test


In [66]:
df_test = df_test.drop(['question1','question2'],axis=1)

In [67]:
df_test.to_csv(data_home+"test_clean_vB.csv",encoding="utf-8", index=False)

In [68]:
## verify if saved correctly
check_test_df = pd.read_csv(data_home+"test_clean_vB.csv",encoding="utf-8")
check_test_df.head()

,test_id,q1_clean_vB,q2_clean_vB
0,0,How does Surface Pro himself 4 compare with iP...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is best way to send money from China ...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,How aberystwyth start reading?,How their can I start reading?


In [69]:
1

1